In [46]:
pip install pyupbit


Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [85]:
import jwt
import hashlib
import os
import requests
import uuid
from urllib.parse import urlencode, unquote
import pyupbit
from datetime import datetime, timedelta
import time
import schedule

class Trade:
    def __init__(self, access_key=None, secret_key=None):
        """
        업비트 API를 활용한 거래 클래스 초기화
        
        Args:
            access_key (str, optional): 업비트 API 액세스 키
            secret_key (str, optional): 업비트 API 시크릿 키
        """
        self.access_key = access_key if access_key else '{ACCESS KEY 입력 : }'
        self.secret_key = secret_key if secret_key else '{SECRET KEY 입력 : }'
        self.server_url = 'https://api.upbit.com'
        self.upbit = pyupbit.Upbit(self.access_key, self.secret_key)
        
        # API 키 확인 및 경고 메시지
        if self.access_key == '{ACCESS KEY 입력 : }' or self.secret_key == '{SECRET KEY 입력 : }':
            print("⚠️ 경고: 실제 API 키가 설정되지 않았습니다. 일부 기능이 제한될 수 있습니다.")
    
    #---------------------------------------
    # API 인증 및 주문 관련 기본 함수
    #---------------------------------------
    
    def orders_status(self, orderid):
        """
        주문 상태 조회
        
        Args:
            orderid (str): 주문 UUID
            
        Returns:
            dict: 주문 상태 정보
        """
        query = {'uuid': f'{orderid}'}
        query_string = urlencode(query).encode()
        m = hashlib.sha512()
        m.update(query_string)
        query_hash = m.hexdigest()

        payload = {
            'access_key': self.access_key,
            'nonce': str(uuid.uuid4()),
            'query_hash': query_hash,
            'query_hash_alg': 'SHA512'
        }
        jwt_token = jwt.encode(payload, self.secret_key)
        authorization = 'Bearer {}'.format(jwt_token)
        headers = {'Authorization': authorization}

        res = requests.get(self.server_url + "/v1/order", params=query, headers=headers)
        return res.json()
    
    def format_order_result(self, order_result):
        """
        업비트 API의 주문 결과(매수/매도)를 보기 좋게 정리해서 출력하는 함수
        
        Args:
            order_result (dict): 업비트 API의 주문 결과 (딕셔너리)
        """
        if not order_result:
            print("주문 결과가 없습니다.")
            return
        
        if isinstance(order_result, str):
            print("오류 발생:", order_result)
            return
        
        # 주문 기본 정보
        is_buy = order_result.get('side') == 'bid'
        order_type = "매수" if is_buy else "매도"
        order_method = {
            'limit': '지정가',
            'market': '시장가',
            'price': '시장가(매수금액 지정)'
        }.get(order_result.get('ord_type'), '알 수 없음')
        
        state_map = {
            'wait': '체결 대기',
            'done': '체결 완료',
            'cancel': '주문 취소'
        }
        state = state_map.get(order_result.get('state'), '알 수 없음')
        
        # 시간 형식 변환
        created_at = order_result.get('created_at', '')
        if created_at:
            try:
                # ISO 시간 포맷에서 시간 추출 ('T'와 '+' 사이)
                time_part = created_at.split('T')[1].split('+')[0]
                date_part = created_at.split('T')[0]
                formatted_time = f"{date_part} {time_part}"
            except:
                formatted_time = created_at
        else:
            formatted_time = "알 수 없음"
        
        # 마켓 정보 추출
        market = order_result.get('market', '알 수 없음')
        currency = market.split('-')[1] if '-' in market else ''
        
        # 수량 및 금액 정보 계산
        volume = float(order_result.get('volume', 0)) if order_result.get('volume') else 0
        executed_volume = float(order_result.get('executed_volume', 0)) if order_result.get('executed_volume') else 0
        execution_percent = (executed_volume / volume * 100) if volume > 0 else 0
        
        # 주문 정보 출력 (매수/매도에 따라 다르게)
        print(f"\n======== {order_type} 주문 결과 요약 ========")
        print(f"주문 종류: {order_type} ({order_method})")
        print(f"주문 상태: {state}")
        print(f"마켓: {market}")
        print(f"주문 시간: {formatted_time}")
        print(f"주문 ID: {order_result.get('uuid', '알 수 없음')}")
        
        # 수량 정보
        print("\n--------- 수량 정보 ---------")
        
        if is_buy and order_result.get('ord_type') == 'price':
            # 시장가 매수(금액 지정)
            print(f"주문 금액: {order_result.get('price', '알 수 없음')} KRW")
            print(f"체결된 수량: {executed_volume} {currency}")
        else:
            # 일반 수량 기반 주문
            print(f"주문 수량: {volume} {currency}")
            print(f"체결된 수량: {executed_volume} {currency} ({execution_percent:.2f}%)")
            print(f"남은 수량: {order_result.get('remaining_volume', '알 수 없음')} {currency}")
        
        # 수수료 정보
        print("\n--------- 수수료 정보 ---------")
        if is_buy:
            print(f"예상 수수료: {order_result.get('reserved_fee', '알 수 없음')} KRW")
            print(f"지불된 수수료: {order_result.get('paid_fee', '알 수 없음')} KRW")
        else:
            print(f"예상 수수료: {order_result.get('reserved_fee', '알 수 없음')} {currency}")
            print(f"지불된 수수료: {order_result.get('paid_fee', '알 수 없음')} {currency}")
        
        # 가격 정보가 있는 경우 (지정가 주문)
        if 'price' in order_result and order_result.get('ord_type') == 'limit':
            print(f"\n지정 가격: {order_result.get('price', '알 수 없음')} KRW")
        
        # 평균 체결 가격 (있는 경우)
        if 'avg_price' in order_result and float(order_result.get('avg_price', 0)) > 0:
            print(f"평균 체결가: {order_result.get('avg_price', '알 수 없음')} KRW")
        
        # 체결 내역이 있는 경우
        trade_count = order_result.get('trades_count', 0)
        if trade_count > 0:
            print(f"\n체결 횟수: {trade_count}회")
        
        print("==============================\n")
    
    #---------------------------------------
    # 조회 관련 함수
    #---------------------------------------
    
    def get_balance(self, ticker):
        """
        특정 코인 잔고 조회
        
        Args:
            ticker (str): 코인 티커 (예: "KRW" 또는 "KRW-BTC"의 "BTC")
            
        Returns:
            float: 보유 수량
        """
        try:
            return self.upbit.get_balance(ticker)
        except Exception as e:
            print(f"잔고 조회 실패: {e}")
            return 0  # 기본값으로 0 반환
    
    def get_current_price(self, ticker):
        """
        특정 코인 현재 시세 조회
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            
        Returns:
            float: 현재 가격
        """
        try:
            return pyupbit.get_current_price(ticker)
        except Exception as e:
            print(f"현재가 조회 실패: {e}")
            return 0
    
    def get_order(self, orderid):
        """
        특정 주문 정보 조회
        
        Args:
            orderid (str): 주문 UUID
            
        Returns:
            dict: 주문 정보
        """
        return self.orders_status(orderid)
    
    def get_ohlcv(self, ticker, interval="day", count=200):
        """
        특정 코인 차트 조회
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            interval (str, optional): 시간 간격 (day, minute1, minute3, minute5, minute10, minute15, minute30, minute60, minute240, week, month)
            count (int, optional): 데이터 개수
            
        Returns:
            pandas.DataFrame: OHLCV 데이터
        """
        try:
            return pyupbit.get_ohlcv(ticker, interval=interval, count=count)
        except Exception as e:
            print(f"OHLCV 조회 실패: {e}")
            return None
    
    def get_market_all(self):
        """
        모든 코인 시세 조회
        
        Returns:
            list: 마켓 정보 리스트
        """
        try:
            url = "https://api.upbit.com/v1/market/all"
            response = requests.get(url)
            return response.json()
        except Exception as e:
            print(f"마켓 정보 조회 실패: {e}")
            return []
    
    def get_krw_markets(self):
        """
        원화 마켓 코인만 조회
        
        Returns:
            list: 원화 마켓 정보 리스트
        """
        try:
            markets = self.get_market_all()
            return [market for market in markets if market['market'].startswith('KRW-')]
        except Exception as e:
            print(f"원화 마켓 조회 실패: {e}")
            return []
    
    #---------------------------------------
    # 주문 관련 함수
    #---------------------------------------
    
    def buy_market_order(self, ticker, amount):
        """
        시장가 매수 주문
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            amount (float): 매수할 금액(KRW)
        
        Returns:
            dict: 주문 결과
        """
        try:
            result = self.upbit.buy_market_order(ticker, amount)
            print(f"시장가 매수 주문: {ticker}, {amount}KRW")
            self.format_order_result(result)  # 주문 결과 포맷팅 출력
            return result
        except Exception as e:
            print(f"시장가 매수 주문 실패: {e}")
            return None
    
    def sell_market_order(self, ticker, volume=None):
        """
        시장가 매도 주문
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            volume (float, optional): 매도할 수량. None이면 전량 매도
        
        Returns:
            dict: 주문 결과
        """
        try:
            if volume is None:
                # 전량 매도
                available_volume = self.upbit.get_balance(ticker)
                if available_volume > 0:
                    result = self.upbit.sell_market_order(ticker, available_volume)
                    print(f"전량 시장가 매도 주문: {ticker}, {available_volume}{ticker.split('-')[1]}")
                    self.format_order_result(result)  # 주문 결과 포맷팅 출력
                    return result
                else:
                    print(f"매도할 {ticker} 수량이 없습니다.")
                    return None
            else:
                # 지정 수량 매도
                result = self.upbit.sell_market_order(ticker, volume)
                print(f"시장가 매도 주문: {ticker}, {volume}{ticker.split('-')[1]}")
                self.format_order_result(result)  # 주문 결과 포맷팅 출력
                return result
        except Exception as e:
            print(f"시장가 매도 주문 실패: {e}")
            return None
    
    def buy_limit_order(self, ticker, price, volume):
        """
        지정가 매수 주문
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            price (float): 매수 희망 가격
            volume (float): 매수 수량
        
        Returns:
            dict: 주문 결과
        """
        try:
            result = self.upbit.buy_limit_order(ticker, price, volume)
            print(f"지정가 매수 주문: {ticker}, 가격: {price}KRW, 수량: {volume}")
            self.format_order_result(result)  # 주문 결과 포맷팅 출력
            return result
        except Exception as e:
            print(f"지정가 매수 주문 실패: {e}")
            return None
    
    def sell_limit_order(self, ticker, price, volume=None):
        """
        지정가 매도 주문
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            price (float): 매도 희망 가격
            volume (float, optional): 매도 수량. None이면 전량 매도
        
        Returns:
            dict: 주문 결과
        """
        try:
            if volume is None:
                # 전량 매도
                available_volume = self.upbit.get_balance(ticker)
                if available_volume > 0:
                    result = self.upbit.sell_limit_order(ticker, price, available_volume)
                    print(f"전량 지정가 매도 주문: {ticker}, 가격: {price}KRW, 수량: {available_volume}")
                    self.format_order_result(result)  # 주문 결과 포맷팅 출력
                    return result
                else:
                    print(f"매도할 {ticker} 수량이 없습니다.")
                    return None
            else:
                # 지정 수량 매도
                result = self.upbit.sell_limit_order(ticker, price, volume)
                print(f"지정가 매도 주문: {ticker}, 가격: {price}KRW, 수량: {volume}")
                self.format_order_result(result)  # 주문 결과 포맷팅 출력
                return result
        except Exception as e:
            print(f"지정가 매도 주문 실패: {e}")
            return None
    
    def cancel_order(self, uuid):
        """
        주문 취소
        
        Args:
            uuid (str): 취소할 주문의 UUID
        
        Returns:
            dict: 취소 결과
        """
        try:
            result = self.upbit.cancel_order(uuid)
            print(f"주문 취소: {uuid}")
            self.format_order_result(result)  # 취소 결과 포맷팅 출력
            return result
        except Exception as e:
            print(f"주문 취소 실패: {e}")
            return None
    
    #---------------------------------------
    # 자동 매매 관련 함수
    #---------------------------------------
    
    def calculate_target_price(self, ticker, k=0.5):
        """
        변동성 돌파 전략의 목표가 계산
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            k (float, optional): 변동성 돌파 계수 (0~1)
            
        Returns:
            float: 목표 매수가
        """
        try:
            df = self.get_ohlcv(ticker, interval="day", count=2)
            if df is None or len(df) < 2:
                return None
                
            prev_range = df['high'].iloc[-2] - df['low'].iloc[-2]
            target_price = df['open'].iloc[-1] + (prev_range * k)
            return target_price
        except Exception as e:
            print(f"목표가 계산 실패: {e}")
            return None
    
    def check_trading_time(self):
        """
        거래 시간 확인 (한국 시간)
        
        Returns:
            tuple: (is_buy_time, is_sell_time) - 매수 가능 시간, 매도 가능 시간 여부
        """
        now = datetime.now()
        # 매수 시간: 09:00 ~ 20:00
        is_buy_time = 9 <= now.hour < 20
        # 매도 시간: 08:50 ~ 09:00
        is_sell_time = (8 == now.hour and now.minute >= 50) or (now.hour == 9 and now.minute < 1)
        return is_buy_time, is_sell_time
    
    def auto_trade(self, ticker, invest_amount, strategy="vb", k=0.5):
        """
        자동 매매 실행
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            invest_amount (float): 투자 금액(KRW)
            strategy (str, optional): 전략 선택 ("vb": 변동성 돌파)
            k (float, optional): 변동성 돌파 전략의 k값
        
        Returns:
            dict: 주문 결과
        """
        try:
            # 현재 시간 확인
            is_buy_time, is_sell_time = self.check_trading_time()
            
            if strategy == "vb":
                # 변동성 돌파 전략
                target_price = self.calculate_target_price(ticker, k)
                if target_price is None:
                    print("목표가 계산 실패")
                    return None
                
                # 현재가 확인
                current_price = self.get_current_price(ticker)
                if current_price == 0:
                    print("현재가 조회 실패")
                    return None
                
                # 매수 조건: 현재가가 목표가 이상이고, 09:00~20:00 사이
                if (current_price >= target_price) and is_buy_time:
                    # 보유 현금 확인
                    krw_balance = self.get_balance("KRW")
                    
                    # 최소 주문 금액 확인 (최소 5000원)
                    order_amount = min(invest_amount, krw_balance)
                    if order_amount >= 5000:
                        return self.buy_market_order(ticker, order_amount)
                    else:
                        print(f"주문 가능 금액이 부족합니다: {order_amount}KRW")
                        return None
                
                # 매도 조건: 08:50~09:00 사이 전량 매도
                elif is_sell_time:
                    return self.sell_market_order(ticker)
                
                else:
                    print(f"현재 매매 조건 미충족: 현재가 {current_price}, 목표가 {target_price}")
                    return None
            else:
                print(f"지원하지 않는 전략입니다: {strategy}")
                return None
            
        except Exception as e:
            print(f"자동 매매 실행 실패: {e}")
            return None
    
    def run(self):
        """
        스케줄러 실행 루프
        """
        while True:
            try:
                schedule.run_pending()
                time.sleep(1)
            except Exception as e:
                print(e)
                time.sleep(1)
    
    def schedule_job(self, job_func, interval=1):
        """
        작업 스케줄링
        
        Args:
            job_func (function): 실행할 함수
            interval (int): 실행 간격(초)
        """
        schedule.every(interval).seconds.do(job_func)
    
    def start(self, job_func=None, interval=1):
        """
        자동 매매 시작
        
        Args:
            job_func (function, optional): 실행할 함수, None이면 기본 작업 사용
            interval (int, optional): 실행 간격(초)
        """
        if job_func:
            self.schedule_job(job_func, interval)
        else:
            # 기본 작업: BTC 자동 매매
            def default_job():
                self.auto_trade("KRW-BTC", 10000)
            
            self.schedule_job(default_job, interval)
        
        print(f"자동 매매 시작 (간격: {interval}초)")
        self.run()
    
    #---------------------------------------
    # 추가 전략 함수
    #---------------------------------------
    
    def vb_strategy(self, ticker, k=0.5):
        """
        변동성 돌파 전략 신호 생성
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            k (float, optional): 변동성 돌파 계수 (0~1)
            
        Returns:
            tuple: (buy_signal, sell_signal, target_price, ma5)
        """
        try:
            df = self.get_ohlcv(ticker, interval="day", count=200)
            if df is None or len(df) < 5:  # 최소 5일치 데이터 필요 (MA5 계산)
                return False, False, None, None
                
            df['range'] = df['high'] - df['low']
            df['target'] = df['open'] + df['range'].shift(1) * k
            df['bull'] = df['open'] > df['target']
            df['ma5'] = df['close'].rolling(window=5).mean()
            df['buy'] = df['bull'] & (df['close'] > df['ma5'])
            df['sell'] = df['bull'] & (df['close'] < df['ma5'])
            
            buy_signal = df['buy'].iloc[-1]
            sell_signal = df['sell'].iloc[-1]
            target_price = df['target'].iloc[-1]
            ma5 = df['ma5'].iloc[-1]
            
            return buy_signal, sell_signal, target_price, ma5
        except Exception as e:
            print(f"전략 계산 실패: {e}")
            return False, False, None, None
    
    def strategy_trade(self, ticker, price, k=0.5):
        """
        변동성 돌파 전략으로 매매 실행
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
            price (float): 거래 금액
            k (float, optional): 변동성 돌파 계수 (0~1)
            
        Returns:
            dict: 주문 결과
        """
        buy_signal, sell_signal, target_price, ma5 = self.vb_strategy(ticker, k)
        
        if buy_signal:
            print(f"매수 신호 발생: {ticker}, 목표가: {target_price}, MA5: {ma5}")
            return self.buy_market_order(ticker, price)
        elif sell_signal:
            print(f"매도 신호 발생: {ticker}, 목표가: {target_price}, MA5: {ma5}")
            return self.sell_market_order(ticker)
        else:
            print(f"매매 신호 없음: {ticker}, 목표가: {target_price}, MA5: {ma5}")
            return None
    
    #---------------------------------------
    # 유틸리티 함수
    #---------------------------------------
    
    def get_daily_report(self, ticker):
        """
        일일 보고서 생성 (현재 상태 요약)
        
        Args:
            ticker (str): 코인 티커 (예: "KRW-BTC")
        """
        try:
            current_price = self.get_current_price(ticker)
            symbol = ticker.split('-')[1]
            balance = self.get_balance(symbol)
            krw_balance = self.get_balance("KRW")
            
            target_price, _, _, ma5 = self.vb_strategy(ticker)
            
            print("\n====== 일일 보고서 ======")
            print(f"현재 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"코인: {ticker}")
            print(f"현재가: {current_price:,} KRW")
            print(f"보유량: {balance} {symbol}")
            print(f"평가금액: {balance * current_price:,.0f} KRW")
            print(f"보유 KRW: {krw_balance:,.0f} KRW")
            print(f"목표가: {target_price:,.0f} KRW")
            print(f"5일 이동평균: {ma5:,.0f} KRW")
            print("========================\n")
        except Exception as e:
            print(f"보고서 생성 실패: {e}")

1. 클래스 인스턴스 생성 및 API 키 입력

In [86]:

access_key = 'ACCESS KEY 입력'
secret_key = 'SECRET KEY 입력'

upbit_bot = Trade(access_key, secret_key)


2. 시장 정보 조회

In [87]:
# 현재가 조회
btc_price = upbit.get_current_price("KRW-BTC")
print(f"비트코인 현재가: {btc_price:,}원")


# 일봉 데이터 조회
btc_daily = upbit.get_ohlcv("KRW-BTC", interval="day", count=10)
print("최근 10일 종가:", btc_daily['close'])

비트코인 현재가: 124,129,000.0원
최근 10일 종가: 2025-03-21 09:00:00    124500000.0
2025-03-22 09:00:00    124150000.0
2025-03-23 09:00:00    126937000.0
2025-03-24 09:00:00    128890000.0
2025-03-25 09:00:00    128823000.0
2025-03-26 09:00:00    128556000.0
2025-03-27 09:00:00    128701000.0
2025-03-28 09:00:00    125800000.0
2025-03-29 09:00:00    123378000.0
2025-03-30 09:00:00    124129000.0
Name: close, dtype: float64


3. 계좌 정보 조회

In [92]:
# 보유 현금(KRW) 조회
krw_balance = upbit_bot.get_balance("KRW")
if krw_balance is None:
    print("보유 KRW: API 키가 설정되지 않았거나 연결 오류가 발생했습니다.")
else:
    print(f"보유 KRW: {krw_balance:,}원")

# 특정 코인 보유량 조회
btc_balance = upbit_bot.get_balance("KRW-BTC")
if btc_balance is None:
    print("보유 BTC: API 키가 설정되지 않았거나 연결 오류가 발생했습니다.")
else:
    print(f"보유 BTC: {btc_balance} BTC")

보유 KRW: 14,861.88938433원
보유 BTC: 0.00012072 BTC


4. 주문 - 시장가 주문

In [ ]:
# 시장가 매수 (5000원어치 비트코인 구매)
#buy_result = upbit_bot.buy_market_order("KRW-BTC", 5000)
#print("매수 결과:", buy_result)

#시장가 매도 (보유 비트코인인 전량 매도)
sell_result = upbit_bot.sell_market_order("KRW-BTC")
print("매도 결과:", sell_result)

전량 시장가 매도 주문: KRW-BTC, 0.00012072BTC

======== 매도 주문 결과 요약 ========
주문 종류: 매도 (시장가)
주문 상태: 체결 대기
마켓: KRW-BTC
주문 시간: 2025-03-30 21:18:03
주문 ID: b5853f30-54ac-4519-af38-58a964f72f15

--------- 수량 정보 ---------
주문 수량: 0.00012072 BTC
체결된 수량: 0.0 BTC (0.00%)
남은 수량: 0.00012072 BTC

--------- 수수료 정보 ---------
예상 수수료: 0 BTC
지불된 수수료: 0 BTC

매도 결과: {'uuid': 'b5853f30-54ac-4519-af38-58a964f72f15', 'side': 'ask', 'ord_type': 'market', 'state': 'wait', 'market': 'KRW-BTC', 'created_at': '2025-03-30T21:18:03+09:00', 'volume': '0.00012072', 'remaining_volume': '0.00012072', 'reserved_fee': '0', 'remaining_fee': '0', 'paid_fee': '0', 'locked': '0.00012072', 'executed_volume': '0', 'trades_count': 0}


RemainingReqParsingError
시장가 매도 주문 실패: '>' not supported between instances of 'NoneType' and 'int'


In [ ]:
#예시

In [ ]:
# 주문 취소 (주문 ID 필요)
if limit_buy and 'uuid' in limit_buy:
    cancel_result = upbit_bot.cancel_order(limit_buy['uuid'])

In [ ]:
# 주문 상태 확인
if buy_result and 'uuid' in buy_result:
    order_status = upbit_bot.get_order(buy_result['uuid'])
    print(order_status)

RemainingReqParsingError
시장가 매도 주문 실패: '>' not supported between instances of 'NoneType' and 'int'
